In [12]:
import numpy as np

In [13]:
np.random.seed(2022)

In [14]:
a = np.random.randint(low=0, high=5, size=10)
a

array([4, 0, 1, 1, 0, 0, 2, 0, 0, 1])

In [15]:
c = np.random.randint(low=0, high=5, size=10)
c

array([1, 3, 3, 3, 0, 3, 0, 3, 0, 0])

In [8]:
import networkx
import torch
import numpy as np
import pandas as pd
from sklearn.metrics import *
from torch_geometric.loader import NeighborSampler, NeighborLoader
from torch_geometric.data import Data, DataLoader
from torch_geometric.nn import GATConv, ResGatedGraphConv, GATv2Conv, SAGEConv, GENConv, DeepGCNLayer, PairNorm, GINConv
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from sklearn.impute import SimpleImputer
import torch.nn.functional as F
from imblearn.under_sampling import RandomUnderSampler
pd.options.mode.use_inf_as_na = True
from collections import Counter
from sklearn.feature_selection import SelectFromModel
import torch.nn as nn
import time
import pickle
from torch.nn import LayerNorm, Linear, ReLU
from torch_scatter import scatter
from tqdm import tqdm
from torch_geometric.loader import RandomNodeSampler
import math
import copy
from sklearn.metrics import f1_score
from torch.optim import lr_scheduler
from torch_geometric.utils import dropout_adj
from sklearn.manifold import TSNE
from torch_geometric.utils import *

In [3]:
from densegat.DenseGAT import DenseGAT

Using TensorFlow backend.


In [5]:
def train():
    total_loss = total_correct = total_examples = 0
    start_time = time.time()
    for batch in train_loader:
        batch = batch.to(device)
        y = batch.y[:batch.batch_size]
        optimizer.zero_grad()
        y_hat = model(batch.x.to(device), batch.edge_index.to(device))[:batch.batch_size]
        loss = F.cross_entropy(y_hat, y)
        loss.backward()
        optimizer.step()
#         scheduler1.step()
        total_loss += float(loss) * batch.batch_size
        total_correct += int((y_hat.argmax(dim=-1) == y).sum())
        total_examples += batch.batch_size
    end_time = time.time()  
    
    return total_loss / total_examples, total_correct / total_examples

def inferrence(model, subgraph_loader):
    total_loss = total_correct = total_examples = 0
    xs = []
    y = []
    pbar = tqdm(total=len(subgraph_loader))
    for batch in subgraph_loader:
        y_hat = model(batch.x, batch.edge_index.to(device))[:batch.batch_size]
        xs.append(y_hat.cpu())
        y_true = batch.y[:batch.batch_size]
        y.append(y_true.cpu())
        pbar.update(1)
    pbar.close()
    y_hat = torch.cat(xs, 0)
    y_hat = y_hat.detach().numpy()
    y_hat = np.argmax(y_hat, -1)
    y = torch.cat(y, 0)
    print(y_hat)
    print(y)
    cr1 = classification_report(y, y_hat,digits=4)
    cf = confusion_matrix(y, y_hat)
    print(cr1)
    print(cf)

def get_metrics(y_true, y_pred):
    cr = classification_report(y_true, y_pred, output_dict=True)
    df = pd.DataFrame(cr).transpose()
    cm = confusion_matrix(y_true, y_pred)
    print(df)
    print(cm)

In [6]:
train_data = torch.load('cicids2017_inducive_train_data')

In [7]:
test_data = torch.load('cicids2017_inducive_test_data')

In [9]:
train_data.edge_index = add_self_loops(train_data.edge_index)[0]

In [10]:
test_data.edge_index = add_self_loops(test_data.edge_index)[0]

In [11]:
train_num_nodes = torch.arange(len(train_data.y))
test_num_nodes = torch.arange(len(test_data.y))

In [13]:
node_num_per_k = 20
depth = 5

In [14]:
hop = [node_num_per_k] * depth
train_loader = NeighborLoader(train_data, input_nodes = train_num_nodes, num_neighbors=hop, batch_size=1024, shuffle=True)

In [15]:
next(iter(train_loader))

Data(x=[64502, 78], edge_index=[2, 1237940], y=[64502], batch_size=1024)

In [16]:
in_channels = train_data.x.size(-1)
num_class = 15
model = DenseGAT(in_channels=in_channels, num_class=num_class, 
                 num_layers=5, num_blocks=1, growth_rate=10, theta=0.8)
loss_all = []

10


In [17]:
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

In [18]:
device = 'cuda:1'
model = model.to(device)

In [19]:
model.train()
epochs = 100
for i in range(1, 1 + epochs):
    train_loader = NeighborLoader(train_data, input_nodes = train_num_nodes, num_neighbors=hop, batch_size=1024, shuffle=True)
    start_time = time.time()
    loss, acc = train()
    end_time = time.time()
    print('epoch: {:04d}'.format(i),
          'loss_train: {:.4f}'.format(loss),
          'acc_train: {:.4f}'.format(acc),
          'time: {:.4f}s'.format(end_time - start_time))
    loss_all.append(loss)

epoch: 0001 loss_train: 0.3189 acc_train: 0.9106 time: 221.0826s
epoch: 0002 loss_train: 0.0887 acc_train: 0.9751 time: 222.3295s
epoch: 0003 loss_train: 0.0669 acc_train: 0.9803 time: 189.5823s
epoch: 0004 loss_train: 0.0584 acc_train: 0.9821 time: 203.1082s
epoch: 0005 loss_train: 0.0509 acc_train: 0.9841 time: 200.3936s
epoch: 0006 loss_train: 0.0425 acc_train: 0.9874 time: 209.1694s


KeyboardInterrupt: 

In [20]:
device = 'cpu'
model = model.to(device)

In [22]:
y_hat = model(test_data.x.to(device), test_data.edge_index.to(device))
y_hat = y_hat.detach().numpy()
y_hat = np.argmax(y_hat, -1)
test_y = test_data.y.numpy()

In [ ]:
get_metrics(test_y, y_hat)